In [ ]:
from openai import OpenAI
import openai
from pydantic import BaseModel
import pandas as pd

# Initialize OpenAI client
client = OpenAI(base_url="https://openrouter.ai/api/v1", api_key="YOUR_API_KEY")

# Define the sentiment response model
class Sentiment(BaseModel):
    sentiment: str
    responsible_area: str

# Function to analyze sentiment with detailed feedback analysis
def provide_sentiment(feedback: str):
    try:
        completion = client.chat.completions.create(
            temperature=0,
            model="google/learnlm-1.5-pro-experimental:free",
            messages=[
                {"role": "system", "content": """You are an assistant that analyzes hotel guest feedback and provides sentiment analysis in JSON format. The sentiment should be Positive, Negative, or Neutral. If the sentiment is Negative, identify the specific area responsible for the feedback, including one or more of the following categories: 'Room Quality', 'Cleanliness', 'Staff Service', 'Food & Beverage', 'Amenities', 'Check-in/Check-out Process', 'Location', 'Pricing', 'Noise', 'Safety', 'Guest Room Comfort', 'Bathroom Facilities', 'Hotel Ambience', 'Parking', 'Eco-friendliness', 'Pet Policy', 'Business Facilities', 'Spa & Wellness', and 'Other'. If no specific area applies, return 'N/A'. Ensure that responses are clear and helpful."""},
                {"role": "user", "content": "Here is a feedback for analysis: \"The hotel room was fantastic, and the staff was friendly!\""},
                {"role": "assistant", "content": "{\"sentiment\": \"Positive\", \"responsible_area\": \"N/A\"}"},
                {"role": "user", "content": "Here is a feedback for analysis: \"The room was dirty, and the staff were rude.\""},
                {"role": "assistant", "content": "{\"sentiment\": \"Negative\", \"responsible_area\": \"Room Quality, Staff Service\"}"},
                {"role": "user", "content": "Here is a feedback for analysis: \"The breakfast was great, but the Wi-Fi was very slow.\""},
                {"role": "assistant", "content": "{\"sentiment\": \"Neutral\", \"responsible_area\": \"N/A\"}"},
                {"role": "user", "content": f"Here is a feedback for analysis: \"{feedback}\""}
            ]
        )

        # Validate and extract the response
        if completion.choices and completion.choices[0].message:
            sentiment_response = completion.choices[0].message.content
            return sentiment_response
        else:
            return "{\"sentiment\": \"Error\", \"responsible_area\": \"N/A\"}"  # Return a default error response if the expected fields are missing

    except Exception as e:
        print(f"Error occurred: {e}")
        return "{\"sentiment\": \"Error\", \"responsible_area\": \"N/A\"}"  # Return a default error response in case of an exception

if __name__ == "__main__":
    # Ask user for feedback input
    feedback = input("Please enter the feedback to be analyzed: ")

    # Analyze the feedback
    sentiment = provide_sentiment(feedback)

    # Print the feedback analysis result
    print("Feedback Analysis Result:", sentiment)


Please enter the feedback to be analyzed: The room had terrible lighting, and the bathroom was not clean. The staff was rude too
Feedback Analysis Result: {"sentiment": "Negative", "responsible_area": ["Room Quality", "Cleanliness", "Bathroom Facilities", "Staff Service"]} 



In [ ]:
from openai import OpenAI
import openai
from pydantic import BaseModel
import pandas as pd

# Initialize OpenAI client
client = OpenAI(base_url="https://openrouter.ai/api/v1", api_key="YOUR_API_KEY")

# Define the sentiment response model
class Sentiment(BaseModel):
    sentiment: str
    responsible_area: str

# Function to analyze sentiment with detailed feedback analysis
def provide_sentiment(feedback: str):
    try:
        completion = client.chat.completions.create(
            temperature=0,
            model="google/learnlm-1.5-pro-experimental:free",
            messages=[
                {"role": "system", "content": """You are an assistant that analyzes hotel guest feedback and provides sentiment analysis in JSON format. The sentiment should be Positive, Negative, or Neutral. If the sentiment is Negative, identify the specific area responsible for the feedback, including one or more of the following categories: 'Room Quality', 'Cleanliness', 'Staff Service', 'Food & Beverage', 'Amenities', 'Check-in/Check-out Process', 'Location', 'Pricing', 'Noise', 'Safety', 'Guest Room Comfort', 'Bathroom Facilities', 'Hotel Ambience', 'Parking', 'Eco-friendliness', 'Pet Policy', 'Business Facilities', 'Spa & Wellness', and 'Other'. If no specific area applies, return 'N/A'. Ensure that responses are clear and helpful."""},
                {"role": "user", "content": f"Here is a feedback for analysis: \"{feedback}\""}
            ]
        )

        # Validate and extract the response
        if completion.choices and completion.choices[0].message:
            sentiment_response = completion.choices[0].message.content
            return sentiment_response
        else:
            return "{\"sentiment\": \"Error\", \"responsible_area\": \"N/A\"}"  # Return a default error response if the expected fields are missing

    except Exception as e:
        print(f"Error occurred: {e}")
        return "{\"sentiment\": \"Error\", \"responsible_area\": \"N/A\"}"  # Return a default error response in case of an exception

# Load the dataset (you can replace this with the actual dataset you are using, e.g., 'feedback.csv')
def analyze_feedback_from_file(file_path: str, feedback_column: str):
    # Load dataset using pandas (CSV or Excel file)
    try:
        # Read dataset (CSV example, you can change this to `pd.read_excel()` if it's an Excel file)
        df = pd.read_csv(file_path)

        # Add new columns to store sentiment analysis results
        df['Sentiment'] = None
        df['Responsible Area'] = None

        # Loop through each feedback entry in the dataset and analyze it
        for index, row in df.iterrows():
            feedback = row[feedback_column]  # Use the provided column name for feedback
            sentiment_analysis = provide_sentiment(feedback)

            # Parse sentiment analysis result from JSON-like string to extract sentiment and responsible area
            if sentiment_analysis.startswith("{") and sentiment_analysis.endswith("}"):
                sentiment_data = eval(sentiment_analysis)  # Safely convert the string to a dictionary
                df.at[index, 'Sentiment'] = sentiment_data.get('sentiment')
                df.at[index, 'Responsible Area'] = sentiment_data.get('responsible_area')
            else:
                df.at[index, 'Sentiment'] = 'Error'
                df.at[index, 'Responsible Area'] = 'N/A'

        # Save the updated dataset to a new CSV file or Excel file
        output_file = "feedback_analysis_output.csv"
        df.to_csv(output_file, index=False)
        print(f"Analysis complete. Results saved to {output_file}")

    except Exception as e:
        print(f"Error occurred while processing the file: {e}")

if __name__ == "__main__":
    # Specify the path to your local dataset (CSV or Excel)
    file_path = input("Enter the path to the feedback dataset (e.g., feedback.csv): ")
    feedback_column = input("Enter the column name containing the feedback (e.g., Review): ")

    # Analyze feedback data from the file
    analyze_feedback_from_file(file_path, feedback_column)


Enter the path to the feedback dataset (e.g., feedback.csv): /content/updated_customer_feedback.csv
Enter the column name containing the feedback (e.g., Review): Reviews
